In [1]:
from transformers import LlamaForCausalLM, LlamaTokenizer
import torch

The parameters

```
padding_side="left", truncation_side="right", add_bos_token=False
```

were taken from the EasyLM llama_model.py code. It is uncertain whether it is the correct configuration, but it seems to run fine for now.


The parameter

```
torch_dtype=torch.float16
```

ensures that 16 bit float precision is used instead of 32 bit, which theoretically reduces the memory requirement to less than 16G of GPU RAM. However, in practice, 
it turned out that sometimes 16G is not sufficient perhaps due to loading the input on CUDA as well, so move up to 2 T4 or increase total GPU RAM by other means to beyond 16G. Alternatively, perhaps there's a way to clear the GPU RAM before generating to make space.

In [2]:
tokenizer = LlamaTokenizer.from_pretrained("/home/jupyter/koala_transformer", device_map="auto", padding_side="left", truncation_side="right", add_bos_token=False)

# full precision - requires slightly more than 32G of GPU RAM in practice
# model = LlamaForCausalLM.from_pretrained("/home/jupyter/koala_transformer", device_map="auto", cache_dir="/home/jupyter/data/transformers")

# half precision - requires slightly more than 16G of GPU RAM in practice.
model = LlamaForCausalLM.from_pretrained("/home/jupyter/koala_transformer", torch_dtype=torch.float16, device_map="auto", cache_dir="/home/jupyter/data/transformers")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The generate function is adapted to the correct format required by https://github.com/young-geng/EasyLM/blob/main/docs/koala.md. Again the parameters

```
do_sample=True, 
top_p=1.0,
num_beams=1,
top_k=50,
```

are taken from the EasyLM code.

The temperature controls how stochastic or creative the model is, and it's a common parameter to adjust.

In [3]:
def generate(prompt, max_new_tokens=1024, temperature=0.1):
    
    # Create the prompt according to the pre-trained format: see https://github.com/young-geng/EasyLM/blob/main/docs/koala.md
    
    template = "BEGINNING OF CONVERSATION: USER: %s "
    input_start = template % prompt
    input_final = template % prompt + "GPT:"
    
    inputs = tokenizer(input_final, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, 
                             do_sample=True, 
                             top_p=1.0,
                             num_beams=1,
                             top_k=50,
                             temperature=temperature)
    result = tokenizer.batch_decode(outputs, skip_special_tokens=True, spaces_between_special_tokens=False)
    
    # Since the task is Autoregressive LM, it's gonna repeat the prompt in the output. Thus, the prompt itself is removed from the output.
    
    return result[0][len(input_start):]

In [4]:
print(generate("How to make a BLT?"))

GPT: A BLT (bacon, lettuce, and tomato) is a classic sandwich that is made with bacon, lettuce, and tomato on a slice of bread. Here's how to make a BLT:

Ingredients:

*   1 slice of bread
*   1/2 cup of bacon, cooked according to your preference (e.g. crispy, medium, or soft)
*   1/2 cup of lettuce
*   1/2 cup of tomato
*   Mayo or other desired condiments
*   Salt and pepper, to taste

Instructions:

1.   Toast the bread to your desired level of crispiness.
2.   Layer the bacon, lettuce, and tomato on the bread.
3.   Spread the mayo or other desired condiments on the bread.
4.   Sprinkle salt and pepper on the bread.
5.   Serve and enjoy!

Note: You can also add other ingredients to the BLT, such as avocado, cucumber, or other vegetables, to make it your own.


In [5]:
print(generate("How to make shrimp tempura?"))

GPT: Shrimp tempura is a delicious Japanese dish made by battering and frying shrimp in a light, crispy batter. Here's how to make shrimp tempura at home:

Ingredients:

*   1 cup flour
*   1 cup cornstarch
*   1 cup water
*   1 cup vegetable oil
*   1 cup shrimp, peeled and deveined
*   Salt and pepper, to taste
*   Batter:
    
    *   1 cup flour
    *   1 cup cornstarch
    *   1 cup water
    *   1 cup vegetable oil
    *   1 cup beer or sake
    *   1 cup soy sauce
    *   1 cup sugar
    *   1 cup dashi (a broth made from bonito flakes)
    
    
    

Instructions:

1.   In a small bowl, combine the flour, cornstarch, water, vegetable oil, beer or sake, soy sauce, and sugar. Mix well to form a smooth batter.
2.   In a large pan, heat the vegetable oil over medium heat. Once the oil is hot, add the shrimp and cook for 1-2 minutes, or until the shrimp are cooked through.
3.   Remove the shrimp from the pan and drain on a paper towel.
4.   In a small bowl, combine the batter and t

In [10]:
article = """Alzheimer’s disease treatment stocks are focused on Alzheimer’s disease, a degenerative brain disorder that results in declining memory and thinking skills and typically affects people in their mid-60s.According to the Alzheimer’s Association, neurons in other areas of the brain also begin to deteriorate as Alzheimer’s disease gets worse, resulting in the loss of basic human functions and overall cognitive impairment.This condition affects more than 6.7 million people in the US alone; it’s also the most common form of dementia and is the sixth leading cause of death in America. Treatments are available to alleviate Alzheimer’s disease symptoms, but because they do not affect the underlying causes of this neurodegenerative disease, they’re only a bandaid solution.Therapies approved by the US Food and Drug Administration (FDA) include: rivastigmine by Novartis (NYSE:NVS); galantamine, developed by Janssen, a division of Johnson & Johnson (NYSE:JNJ); donepezil by Pfizer (NYSE:PFE); and AbbVie's (NYSE:ABBV) memantine.Since there is no cure for Alzheimer’s disease, death is often the result for patients as the ailment devastates the brain. And unfortunately, Alzheimer’s disease is rising in prevalence — a report from Research and Markets suggests that the global Alzheimer’s disease treatment market will be worth a significant US$9.64 billion by 2028 as more patients need treatment, and as more investments are made in biomarkers for diagnosis and drug development.Here the Investing News Network takes a look at a few of the biggest Alzheimer’s disease treatment stocks on the NASDAQ. Companies are listed in order of market cap from largest to smallest, and all data was current as of April 19, 2023.. Biogen (NASDAQ:BIIB) Company Profile.Market cap: US$42.27 billion; current share price: US$292.80.Massachusetts-based Biogen, a pioneer in the field of neuroscience, is developing, manufacturing and marketing therapies aimed at treating serious neurological, neurodegenerative, autoimmune and rare diseases.The global biotechnology firm’s research areas include Alzheimer's disease and dementia. However, Biogen’s launch of its FDA-approved Alzheimer’s disease drug Aduhelm faced a lot of pushback in 2022, both from the market and from Congress, over what was viewed as a hasty fast-track approval process and exorbitant costs to patients.In early 2023, Biogen is looking to give it another go, this time with LEQEMBI (lecanemab-irmb) for the treatment of Alzheimer's disease, which the FDA has approved under its accelerated approval pathway. The drug was jointly developed by Biogen and Tokyo-based pharmaceutical company Eisai (OTC Pink:ESALF,TSE:4523). Intra-Cellular Therapies (NASDAQ:ITCI).Company Profile.Market cap: US$5.96 billion; current share price: US$62.59.As its name suggests, biopharma company Intra-Cellular Therapies is taking an intracellular approach to developing therapies for patients with complex psychiatric and neurologic diseases, including Parkinson’s disease and Alzheimer’s disease.Intra-Cellular Therapies is currently evaluating ITI-1284-ODT-SL, a deuterated form of lumateperone that comes in an oral disintegrating tablet; ITI-1284-ODT-SL is in Phase 1 studies, including drug-drug interaction studies. The company expects to commence Phase 2 clinical trials in agitation in patients with probable Alzheimer’s disease in 2023. 3. Axsome Therapeutics (NASDAQ:AXSM).Company Profile.Market cap: US$3.16 billion; current share price: US$72.64.New York-based Axsome Therapeutics is developing and commercializing a portfolio of novel therapies for disorders of the central nervous system with limited treatment options. Products currently on the market from the biopharmaceutical company include Auvelity, a rapid-acting oral antidepressant for the treatment of major depressive disorder; and Sunosi, a once-daily prescription medicine to treat excessive daytime sleepiness due to obstructive sleep apnea or narcolepsy.Axsome’s product pipeline includes AXS-05, a NMDA receptor antagonist with multimodal activity. It has breakthrough therapy designation from the FDA and is currently in Phase 3 clinical studies for the treatment of Alzheimer’s disease agitation. """

response = generate("You are an equity researcher specialized in the health and pharmaceutical industry. Write a summary for the following article. Focus on the information that will impact investment decisions on the companies mentioned.\n" + article)
print(response)

GPT: The article discusses the treatment options for Alzheimer's disease and the companies that are developing therapies for this condition. Biogen, Intra-Cellular Therapies, and Axsome Therapeutics are three companies that are currently developing therapies for Alzheimer's disease. Biogen's drug Aduhelm faced pushback in 2022, but the company is planning to give it another go with its new drug LEQEMBI. Intra-Cellular Therapies is currently evaluating ITI-1284-ODT-SL, a deuterated form of lumateperone that comes in an oral disintegrating tablet. Axsome Therapeutics is developing and commercializing a portfolio of novel therapies for disorders of the central nervous system with limited treatment options. Its product pipeline includes AXS-05, a NMDA receptor antagonist with multimodal activity that is currently in Phase 3 clinical studies for the treatment of Alzheimer's disease agitation.
